In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
%matplotlib inline

In [27]:
df = pd.read_excel('/content/Tweets.xlsx')
df.head()

,tweet_id,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [28]:
df.isnull().sum()

tweet_id              0
airline               0
name                  0
retweet_count         0
text                  0
tweet_coord       13621
tweet_created         0
tweet_location     4733
user_timezone      4820
dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 9 columns):
tweet_id          14640 non-null int64
airline           14640 non-null object
name              14640 non-null object
retweet_count     14640 non-null int64
text              14640 non-null object
tweet_coord       1019 non-null object
tweet_created     14640 non-null object
tweet_location    9907 non-null object
user_timezone     9820 non-null object
dtypes: int64(2), object(7)
memory usage: 1.0+ MB


In [0]:
import re
from re import sub

def text_to_word_list(text):

    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    # text = text.split()

    return text

In [49]:
df.text = df.text.apply(lambda x: text_to_word_list(x))
df.head()

,tweet_id,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,Virgin America,cairdin,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,Virgin America,jnardino,0,virginamerica plus you ve added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,Virgin America,yvonnalynn,0,virginamerica i didn t today must mean i need...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,Virgin America,jnardino,0,virginamerica it s really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,Virgin America,jnardino,0,virginamerica and it s a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [50]:
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
for i in df.text:
  for j in i:
    lemmatizer.lemmatize(j)

In [52]:
df.head()

,tweet_id,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,Virgin America,cairdin,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,Virgin America,jnardino,0,virginamerica plus you ve added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,Virgin America,yvonnalynn,0,virginamerica i didn t today must mean i need...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,Virgin America,jnardino,0,virginamerica it s really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,Virgin America,jnardino,0,virginamerica and it s a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidfVec = TfidfVectorizer(stop_words='english')

In [0]:
tfs = tfidfVec.fit_transform(df.text)

In [58]:
print(tfidfVec.get_feature_names()[:10])

['aa', 'aaaand', 'aaadvantage', 'aaalwayslate', 'aacustomerservice', 'aadavantage', 'aadelay', 'aadv', 'aadvantage', 'aafail']


In [57]:
len(tfidfVec.get_feature_names())

13354

In [59]:
tfs.shape

(14640, 13354)

In [0]:
from sklearn.cluster import KMeans

In [0]:
kmeans  = KMeans(n_clusters=2).fit(tfs)

In [67]:
kmeans.labels_

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [68]:
df['labels'] = kmeans.labels_
df.head()

,tweet_id,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,labels
0,570306133677760513,Virgin America,cairdin,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1
1,570301130888122368,Virgin America,jnardino,0,virginamerica plus you ve added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),1
2,570301083672813571,Virgin America,yvonnalynn,0,virginamerica i didn t today must mean i need...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1
3,570301031407624196,Virgin America,jnardino,0,virginamerica it s really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),1
4,570300817074462722,Virgin America,jnardino,0,virginamerica and it s a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),1


In [69]:
df.nunique()

tweet_id          14485
airline               6
name               7701
retweet_count        18
text              14392
tweet_coord         832
tweet_created     14247
tweet_location     3022
user_timezone        85
labels                2
dtype: int64

In [72]:
df['labels'] = df['labels'].apply(lambda x : 'Positive' if x == 1 else 'Negative')
df.head()

,tweet_id,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,labels
0,570306133677760513,Virgin America,cairdin,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),Positive
1,570301130888122368,Virgin America,jnardino,0,virginamerica plus you ve added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),Positive
2,570301083672813571,Virgin America,yvonnalynn,0,virginamerica i didn t today must mean i need...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),Positive
3,570301031407624196,Virgin America,jnardino,0,virginamerica it s really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),Positive
4,570300817074462722,Virgin America,jnardino,0,virginamerica and it s a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),Positive


In [74]:
df['labels'].value_counts()

Positive    12405
Negative     2235
Name: labels, dtype: int64

In [0]:
df.to_csv('Sentiment_analysed_file.csv')